# 🦜🔗 LangChain核心源代码解读：关于LLM和Agent（下）

# 课程开始

## 下半场的闲聊

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)
- 🌹 [Chain 遗产](https://python.langchain.com/docs/modules/chains)

### 🌹 智能体相关类整理

**（1）智能体类型**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（`ReAct`的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用`search 工具`不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（`ReAct`，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（`ReAct`，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持`OpenAI Function Calling`）
    - OPENAI_MULTI_FUNCTIONS（支持`OpenAI Function Calling`，支持多函数调度）

----
**（2）智能体执行器子组件：**

[查看 langchain_core/agents.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/core/langchain_core/agents.py)

- `AgentAction`
    - `AgentActionMessageLog`
- `AgentStep`
- `AgentFinish`

----
**（3）单动智能体：**
- `BaseSingleActionAgent`
    - RunnableAgent
    - LLMSingleActionAgent（__deprecated__：`create_***_agent`）
    - XMLAgent（__deprecated__：create_xml_agent）
    - Agent
        - ChatAgent（__deprecated__：`create_react_agent`）
        - ConversationalAgent（__deprecated__：`create_react_agent`）
        - ConversationalChatAgent（__deprecated__：`create_json_chat_agent`）
        - StructuredChatAgent（__deprecated__：`create_structured_chat_agent`）
        - ZeroShotAgent（__deprecated__：`create_react_agent`）
        - ReActDocstoreAgent（__deprecated__）
            - ReActTextWorldAgent（__deprecated__）
        - SelfAskWithSearchAgent（__deprecated__：`create_self_ask_with_search`）
    - OpenAIFunctionsAgent（__deprecated__：`create_openai_functions_agent`）

----
**（4）多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__deprecated__：`create_openai_tools_agent`）

----
**（5）Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - `AgentExecutor`
                - MRKLChain（__deprecated__）
                - ReActChain（__deprecated__）
                - SelfAskWithSearchChain（__deprecated__）
        - `OpenAIAssistantRunnable`

----
**（6）提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate [Dict, PromptValue]
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - `AgentScratchPadChatPromptTemplate`

----
**（7）输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - `AgentOutputParser`
            - `MultiActionAgentOutputParser`

### 🌹 从源码中看 openai 作为先驱的影响

- [partners/openai/langchain_openai/chat_models/base.py（OpenAI模型）](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/chat_models/base.py)
- [community/langchain_community/adapters/openai.py（OpenAI风格API转换）](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/adapters/openai.py)
- [core/langchain_core/utils/function_calling.py（OpenAI风格回调工具函数）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- [core/langchain_core/output_parsers/openai_functions.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_functions.py)

# （二）解读源码，拆解和自定义智能体

<div class="alert alert-info">
    <b>干货从这里开始！</b><br>
    接下来的例子中，会穿插 langchian 源码解读。
</div>

In [120]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 4、解读 OpenAI 工具回调风格智能体

### ✍️ 拆解 OpenAI 工具回调智能体的定义过程

![还是那个故事，但这次我们让老大爷聪明一点...如果可以的话。](./madongmei.gif)

In [119]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """想问你找的人住哪里就问我吧，我是楼下老大爷"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

def create_openai_neighber(llm):
    """
    使用openai智能体定义一个应用
    """
    # 定义 tools
    tools = [ask_neighber]
    # 定义 prompt
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return executor

neighber = create_openai_neighber(ChatZhipuAI())

### 🌹 prompt： 观察 OpenAI 智能体的提示语

#### （1）从 Langsmith 的 hub 下载 hwchase17/openai-tools-agent

[查看 hub.pull("hwchase17/openai-tools-agent")](https://smith.langchain.com/hub/hwchase17/openai-tools-agent)

#### （2）等价的自定义 Prompt 模板

In [50]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [58]:
prompt.invoke({
    "input": "请问马冬梅的家在哪里？",
    "chat_history": [],
    "agent_scratchpad": []
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='请问马冬梅的家在哪里？')])

### 🌹 agent：阅读 create_openai_tools_agent 源码

[查看 langchain/agents/openai_tools/base.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)

```python
def create_openai_tools_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """Create an agent that uses OpenAI tools."""

    missing_vars = {"agent_scratchpad"}.difference(prompt.input_variables)
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent
```

### 🌹 executor： 阅读 AgentExcutor 源码

[查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L915)

- 代码中的运行逻辑非常复杂，因此我们可以通过下面的方式研究其实际运行过程。
- 作为 Chain 子类，AgentExcutor 覆写了 stream 方法。

### 🌹 run：观察 OpenAI 智能体的运行过程

#### （1）简单执行：invoke

In [121]:
# invoke
neighber.invoke({"input":"马冬梅住哪里"})



> Entering new AgentExecutor chain...

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322根据楼下老大爷的回答，马冬梅住在楼上322。

> Finished chain.


{'input': '马冬梅住哪里', 'output': '根据楼下老大爷的回答，马冬梅住在楼上322。'}

#### （2）流输出：stream（仅智能体中的流）

In [114]:
# stream
for s in neighber.stream({"input":"马冬梅住哪里"}):
    print(s)



> Entering new AgentExecutor chain...
{'actions': [OpenAIToolAgentAction(tool='ask_neighber', tool_input={'query': '马冬梅住哪里'}, log="\nInvoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516974814413003915', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})], tool_call_id='call_8516974814413003915')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516974814413003915', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})]}

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322{'steps': [AgentStep(action=OpenAIToolAgentAction(tool='ask_neighber', tool_input={'query': '马冬梅住哪里'}, log="\nInvoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '

#### （3）还是要使用事件流：astream_events

**看看方法 astream_events 的能力：**

In [14]:
# astream_events
async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print(e['name'], e['tags'], e['event'])

AgentExecutor

> Entering new AgentExecutor chain...
 [] on_chain_start
RunnableSequence [] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_stream
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableLambda ['map:key:agent_scratchpad'] on_chain_end
RunnableParallel<agent_scratchpad> [] on_chain_end
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_end
ChatPromptTemplate ['seq:step:2'] on_prompt_start
ChatPromptTemplate ['seq:step:2'] on_prompt_end
ChatZhipuAI ['seq:step:3'] on_chat_model_start
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_end
OpenAIToolsAgentOutpu

#### （4）观察与大模型的交互过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [34]:
from langchain_zhipu import ChatZhipuAI
neighber = create_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)



> Entering new AgentExecutor chain...

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬梅住哪里')]]}

 ---------- ChatZhipuAI -- on_chat_model_end ----------
OUTPUT:
{'generations': [[{'text': '', 'generation_info': {'finish_reason': 'tool_calls'}, 'type': 'ChatGenerationChunk', 'message': AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516985431572319825', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})}]], 'llm_output': None, 'run': None}

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322
 ---------- ask_neighber -- on_tool_end ----------
INPUT:
{'query': '马冬梅住哪里'}

 ---------- ask_neighber -- on_tool_end ----------
OUTPUT:
楼上322

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬

#### （5）总结智能体的定义过程

- STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求
- STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息
- STEP-3 调用本地工具（langchain）: 调用工具
- STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果
- STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

### ✍️ 一步一步执行：就像我们常对大模型说的那样

#### （1）STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求

In [97]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

input = "马冬梅住在哪里？"
history = []

llm = ChatZhipuAI(tools=[convert_to_openai_tool(ask_neighber)])

agent = prompt | llm

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    <b>convert_to_openai_function</b> 和 <b>convert_to_openai_tool</b> 的区别是什么？
</div>

**🌞 参考：**
- [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


In [38]:
convert_to_openai_function(ask_neighber)

{'name': 'ask_neighber',
 'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [39]:
convert_to_openai_tool(ask_neighber)

{'type': 'function',
 'function': {'name': 'ask_neighber',
  'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

#### （2）STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息

In [104]:
steps_info = []
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]})

#### （3）STEP-3 调用本地工具（langchain）: 调用工具

In [105]:
resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"]

'{"query":"马冬梅住在哪里？"}'

In [107]:
import json
tool_args = json.loads(resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"])
tool_args

{'query': '马冬梅住在哪里？'}

In [108]:
resp_tool = ask_neighber.invoke(tool_args)
resp_tool

'楼上322'

#### （4）STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果

In [110]:
steps_info.append(input)
steps_info.append(resp_llm)
steps_info.append(resp_tool)
steps_info

['马冬梅住在哪里？',
 '楼上322',
 '马冬梅住在哪里？',
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]}),
 '楼上322']

#### （5）STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [111]:
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='根据我的查询结果，马冬梅住在楼上322。您还需要我提供其他信息吗？')

## 5、解读 ReAct 风格智能体

### 🌹 构造 ReAct 智能体的提示语

[查看 hub.pull("hwchase17/react")](https://smith.langchain.com/hub/hwchase17/react)

In [149]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

In [150]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### ✍️ 拆解 react 智能体的核心结构

#### （1）定义 ReAct 智能体

In [160]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain.tools import tool
import re

@tool
def ask_neighber(query: str) -> str:
    """我是马冬梅的邻居老大爷，关于她的事情你可以问我"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

prompt_react = """
请尽你最大努力回答用户的问题，并使用中文。
你可以访问如下工具：
{tools}

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是 {tool_names} 等其中之一
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: {input}
Thought:{agent_scratchpad}
"""

def create_react_neighber(llm):
    # 定义 tools
    tools = [ask_neighber]
    # 定义 prompt
    prompt = PromptTemplate.from_template(prompt_react)
    # 定义 Agent
    agent = create_react_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    return executor

neighber = create_react_neighber(ChatZhipuAI())

#### （2）简单执行：invoke

In [161]:
# invoke
neighber.invoke({"input":"马冬梅住哪里"})



> Entering new AgentExecutor chain...
我需要获取马冬梅的住址信息，这个问题明显需要使用提供的工具来询问她的邻居。

Action: ask_neighber
Action Input: '马冬梅住在哪里？'
Observation楼上322Thought: 通过邻居老大爷的回答，我已经得到了马冬梅的住址信息。

Final Answer: 马冬梅住在楼上322。

> Finished chain.


{'input': '马冬梅住哪里', 'output': '马冬梅住在楼上322。'}

#### （3）大模型的流输出：stream VS astream_events

In [164]:
for chunk in neighber.stream({"input": "马冬梅住哪里"}):
    print(chunk)



> Entering new AgentExecutor chain...
{'actions': [AgentAction(tool='ask_neighber', tool_input='马冬梅住在哪里？"\nObservation', log='我需要获取马冬梅的住址信息，这个问题明显需要使用提供的工具 ask_neighber 来向她的邻居老大爷询问。\n\nAction: ask_neighber\nAction Input: "马冬梅住在哪里？"\nObservation')], 'messages': [AIMessage(content='我需要获取马冬梅的住址信息，这个问题明显需要使用提供的工具 ask_neighber 来向她的邻居老大爷询问。\n\nAction: ask_neighber\nAction Input: "马冬梅住在哪里？"\nObservation')]}
我需要获取马冬梅的住址信息，这个问题明显需要使用提供的工具 ask_neighber 来向她的邻居老大爷询问。

Action: ask_neighber
Action Input: "马冬梅住在哪里？"
Observation楼上322{'steps': [AgentStep(action=AgentAction(tool='ask_neighber', tool_input='马冬梅住在哪里？"\nObservation', log='我需要获取马冬梅的住址信息，这个问题明显需要使用提供的工具 ask_neighber 来向她的邻居老大爷询问。\n\nAction: ask_neighber\nAction Input: "马冬梅住在哪里？"\nObservation'), observation='楼上322')], 'messages': [HumanMessage(content='楼上322')]}
Thought: 邻居老大爷已经告诉我马冬梅的住址是在楼上322，我现在知道问题的答案了。

Final Answer: 马冬梅住在楼上322。

> Finished chain.
{'output': '马冬梅住在楼上322。', 'messages': [AIMessage(content='Thought: 邻居老大爷已经告诉我马冬梅的住址是在楼上322

In [167]:
async for event in neighber.astream_events({"input": "马冬梅住哪里"}, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="_")
    else:
        print(kind)



> Entering new AgentExecutor chain...
on_chain_start
on_chain_start
on_chain_start
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_stream
on_chain_stream
on_chain_end
on_chain_end
on_chain_end
on_prompt_start
on_prompt_end
on_chat_model_start
我_需要_获取_马_冬_梅_的_住_址_信息_，_这个问题_明显_需要_使用_提供_的信息_查询_工具_。

Action_:_ ask__ne_igh_ber_
Action_ Input_:_ '_马_冬_梅_住_在哪里_'
Observ_ation__on_chat_model_end
on_parser_start
on_parser_end
on_chain_stream
on_chain_end
on_chain_stream
我需要获取马冬梅的住址信息，这个问题明显需要使用提供的信息查询工具。

Action: ask_neighber
Action Input: '马冬梅住在哪里'
Observationon_tool_start
楼上322on_tool_end
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_stream
on_chain_stream
on_chain_end
on_chain_end
on_chain_end
on_prompt_start
on_prompt_end
on_chat_model_start
Thought_:_ 我_已经_从_邻居_老大_爷_那里_得到了_马_冬_梅_的_住_址_信息_。

Final_ Answer_:_ 马_冬_梅_住在_楼上_3_22_。__on_chat_model_end
on_parser_start
on_parser_end
on_chain_stream
on_cha

### 🌹 观察 ReAct 智能体的运行过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [159]:
from langchain_zhipu import ChatZhipuAI
neighber = create_react_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print_line = lambda : print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            input = e['data']['input']
            print_line()
            print("INPUT:")
            if("messages" in input):
                print(input["messages"][0][0].content)
            else:
                print(input)
        if("output" in e['data']):
            output = e['data']['output'] 
            print_line()
            print("OUTPUT:")
            if("generations" in output):
                print(output["generations"][0][0]["text"])
            else:
                print(output)
        # print("\n", e)



> Entering new AgentExecutor chain...

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:

请尽你最大努力回答用户的问题，并用中文回答问题。
你可以访问如下工具：
ask_neighber: ask_neighber(query: str) -> str - 我是马冬梅的邻居老大爷，关于她的事情你可以问我

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是 ask_neighber 等其中之一
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: 马冬梅住哪里
Thought:


 ---------- ChatZhipuAI -- on_chat_model_end ----------
OUTPUT:
我需要获取马冬梅的住址信息，这个问题明显需要使用邻居老大爷提供的询问功能。

Action: ask_neighber
Action Input: '马冬梅住在哪里？'
Observation
我需要获取马冬梅的住址信息，这个问题明显需要使用邻居老大爷提供的询问功能。

Action: ask_neighber
Action Input: '马冬梅住在哪里？'
Observation楼上322
 ---------- ask_neighber -- on_tool_end ----------
INPUT:
'马冬梅住在哪里？'
Observation

 ---------- ask_neighber -- on_tool_end ----------
OUTPUT:
楼上322

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:

请尽你最大努力回答用户的问题，并用中文回答问题。
你可以访问如下工具：


### 🌹 阅读源码，解读 ReAct 智能体

- `ReAct` 比 `OpenAI智能体` 多了必要的解析过程。

- [查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)
- [查看 ReAct 输出解析的源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py#L22-L95)

<div class="alert alert-warning">
    <b>思考：上面的提示语中为什么不修改这几个关键词？</b><br>
    <ul>
        <li>Final Answer</li>
        <li>Action</li>
        <li>Action Input</li>
        <li>Observation</li>
    </ul>
</div>

## 6、在LCEL框架下重定义《手撕AutoGPT》中的智能体

### ✍️ 自定义基于思维链的提示语

In [174]:
PROMPT_COT = """
你是强大的AI助手，可以使用工具与指令自动化解决问题。

你必须遵循以下约束来完成任务:
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 如果你已经完成所有任务，确保以"FINISH"指令结束。
5. 用中文思考和输出。
6. 如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。DO NOT MAKE UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作做中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的准确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保该分析或计算中涉及的所有子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的。
15. 不要向用户提问。

你的任务是:
{input}

你有非常优秀的逻辑分析能力，可以通过因果关系找到最优的解决方案。

你要参考之前的思考记录:
{agent_scratchpad}

你需要评估你的表现:
1. 尽你最大的努力，用你最好的水平，通过分析和检查，做出最好的决定。
2. 带着全局观，自我反思你计划与动作。
3. 考虑你之前的策略与决策来改善的你的计划。
4. 如果你反复得到相同的结果，修改你的计划和决策，避免死循环。
5. 如果你当前的动作无法获取到需要的信息，尝试展开关键概念的定义，再重新推理。

如果你必须选择工具才能完成任务，可以使用以下工具之一，它们又称为动作或actions:

{tools}

你有两种情况你都应当选择FINISH工具:

(1) 如果任务显示“无”、“没有了”、“已完成”、“FINISH”或类似表达，你就选择FINISH工具，
(2) 如果没有选择任何工具就获得了输出，你就直接选择FINISH工具

无论上述哪一种情况，你都必须将最终结果以键值对的方式输出给FINISH工具，其中键的名称是"output"，值就是你的最终结果。

你必须根据以下格式说明，输出你的思考过程:
1. 关键概念: 任务中涉及的组合型概念或实体。已经明确获得取值的关键概念，将其取值完整备注在概念后。
2. 概念拆解: 将任务中的关键概念拆解为一系列待查询的子要素。每个关键概念一行，后接这个概念的子要素，每个子要素一行，行前以' -'开始。
3. 反思:
   - 自我反思，观察以前的执行记录，思考概念拆解是否完整、准确。
   - 一步步思考是否每一个的关键概念或要素的查询都得到了准确的结果。
   - 反思你已经得到哪个要素/概念。你得到的要素/概念取值是否正确。从当前的信息中还不能得到哪些要素/概念。
   - 每个反思一行，行前以' -'开始。
4. 思考: 观察执行记录和你的自我反思，并一步步思考
  （1）分析要素间的依赖关系，例如：
    i. 我是否需要先获得A的值/定义，才能通过A来获得B？
    ii. 如果我先获得A，是否可以通过A筛选B，减少穷举每个B的代价？
    iii. A和B是否存在在同一数据源中，我能否在获取A的同时获取B？
    iv. 是否还有更高效或更聪明的办法来查询一个概念或要素？
    v. 如果上一次尝试查询一个概念或要素时失败了，我是否可以尝试从另一个资源中再次查询？
    vi. 诸如此类，你可以扩展更多的思考 ...
  （2）根据以上分析，排列子要素间的查询优先级
  （3）找出当前需要获得取值的子要素
  注意，不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源！
5. 推理: 根据你的反思与思考，一步步推理被选择的子要素取值的获取方式。如果前一次的计划失败了，请检查输入中是否包含每个概念/要素的明确定义，并尝试细化你的查询描述。
6. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）一步步分析，包括数据源，对数据源的操作方式，对数据的分析方法。有哪些已知常量可以直接代入此次分析。
  （3）不要尝试计算文件的每一个元素，这种计算代价太高，是严格禁止的。你可以通过分析找到更有效的方法，比如条件筛选。
  （4）上述分析是否依赖某个要素的取值/定义，且该要素的取值/定义尚未获得。若果是，重新规划当前动作，确保所有依赖的要素的取值/定义都已经获得。
  （5）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （6）确保你执行的动作涉及的所有要素都已获得确切的取值/定义。
  （7）如果全部子任务已完成，请用FINISH动作结束任务。

你必须根据以下格式说明，输出所选择执行的动作/工具/指令:
{action_format_instructions}
"""

### ✍️ 自定义 OutputParser

In [175]:
from typing import List, Optional, Dict, Any, Union, Callable
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.output_parsers import PydanticOutputParser

In [176]:
class Action(BaseModel):
    name: str = Field(
        description="The name of tool or action: FINISH or Other tool names."
        )
    args: Optional[Dict[str, Any]] = Field(
        default=None,
        description="Parameters of tool or action are composed of names and values."
        )

# 解析Action
_action_output_parser = PydanticOutputParser(pydantic_object=Action)
_action_parser_format = _action_output_parser.get_format_instructions()

class ReasonOutputParser(AgentOutputParser):
    """解析单个动作的智能体action和输入参数。
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        action: Action = _action_output_parser.invoke(text)
        name: Optional[str] = action.name
        args: Optional[Dict[str, Any]] = action.args if text is not None else "No Args"
        log: str = text if text is not None else ""

        if name == "FINISH":
            return AgentFinish(args, log)
        elif name is not None:
            return AgentAction(name, args, log)

    @property
    def _type(self) -> str:
        return "Chain-of-Thought"


### ✍️ 自定义一个智能体

In [177]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

def _prompt_creator(prompt: str) -> Callable[[List[str]], str]:
    def creator(tools: List[str]) -> str:
        # 请注意，智谱AI等国内大模型对于pydantic的参数解析并不友好，使用JSON描述参数时会误读
        # 因此，不要使用 render_text_description_and_args 来生成工具描述
        tools_format = render_text_description(tools)

        template = PromptTemplate.from_template(prompt)
        return template.partial(
            tools=tools_format,
            action_format_instructions=_action_parser_format,
        )

    return creator

# 基于 CoT 的智能体
def create_reason_agent(llm: Any, prompt: Optional[str] = None, tools: List[str] = []) -> Any:
    prompt_creator = _prompt_creator(PROMPT_COT)
    if prompt is not None:
        prompt_creator = _prompt_creator(prompt)

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"])
        )
        | prompt_creator(tools)
        | llm
        | ReasonOutputParser()
    )

    return agent

### ✍️ 尝试运行

In [178]:
def create_cot_neighber(llm):
    # 定义 tools
    tools = [ask_neighber]
    # 定义 Agent
    agent = create_reason_agent(llm, tools=tools)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    return executor

neighber = create_cot_neighber(ChatZhipuAI())

In [179]:
async for chunk in neighber.astream_events({"input": "马冬梅家住哪里？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)



> Entering new AgentExecutor chain...
关键_概念_:
-_ 马_冬_梅_住_址_

概念_拆_解_:
-_ 马_冬_梅_住_址_
 _ -_ -_ 马_冬_梅_的_邻居_信息_

反思_:
-_ -_ 马_冬_梅_的_住_址_需要_通过_询问_邻居_来_获取_，_目前_没有_其他_方式_获得_此_信息_。

思考_:
-_ 根_据_给定_工具_，_我_可以通过_询问_邻居_来_获取_马_冬_梅_的_住_址_信息_。_这是_目前_唯一_可_用的_信息_源_。

推理_:
-_ 既然_马_冬_梅_的_邻居_愿意_提供_关于_她的_信息_，_我将_使用_工具_来_询问_她的_住_址_。

计划_:
-_ 使用_工具_ "_ask__ne_igh_ber_"_ 来_获取_马_冬_梅_的_住_址_。

所_选择_执行_的动作_/_工具_/_指令_:

```_json_
{"_name_":_ "_ask__ne_igh_ber_",_ "_args_":_ {"_query_":_ "_马_冬_梅_住_在哪里_？_"}}
```__关键概念:
- 马冬梅住址

概念拆解:
- 马冬梅住址
  - - 马冬梅的邻居信息

反思:
- - 马冬梅的住址需要通过询问邻居来获取，目前没有其他方式获得此信息。

思考:
- 根据给定工具，我可以通过询问邻居来获取马冬梅的住址信息。这是目前唯一可用的信息源。

推理:
- 既然马冬梅的邻居愿意提供关于她的信息，我将使用工具来询问她的住址。

计划:
- 使用工具 "ask_neighber" 来获取马冬梅的住址。

所选择执行的动作/工具/指令:

```json
{"name": "ask_neighber", "args": {"query": "马冬梅住在哪里？"}}
```楼上322关键_概念_:_ 马_冬_梅_住_址_

概念_拆_解_:
-_ 马_冬_梅_住_址_
 _ -_ -_ 马_冬_梅_的_邻居_信息_

反思_:
-_ -_ 已_通过_询问_邻居_得到_马_冬_梅_的_住_址_信息_。

思考_:
-_ 所有_关键_概念的_子_要素_都已_查询_到_结果_，_马_冬_梅_的_住_址_已经_明确_。

推理_:
-_ 马_冬_梅_的_住_址_已经_通过_邻居_获得_，_无需_

# 结束了

## ❤️ Langgraph中的惊喜

**一些较新的智能体论文实践...**

- [Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb)
- [Hierarchical Agent Teams](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb)
- [Multi Agent Collaboration](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/multi-agent-collaboration.ipynb)
- [Plan And Execute](https://github.com/langchain-ai/langgraph/blob/main/examples/plan-and-execute/plan-and-execute.ipynb)
- [ReWoo](https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb)
- [Reflexion](https://github.com/langchain-ai/langgraph/blob/main/examples/reflexion/reflexion.ipynb)
- [Self Discover](https://github.com/langchain-ai/langgraph/blob/main/examples/self-discover/self-discover.ipynb)
- ...

## ❤️ 总结

- 我们一起解构了 langchain 部份组件的源码结构
    - Runnable
    - BaseLanguageModel / BaseChatModel
    - create_openai_tools_agent
    - create_react_agent
    - Chain
    - AgentExecutor
- 我们从零开始集成了达模型到 langchain
- 我们深度解构了 OpenAI / ReAct 智能体的细节
- 我们自定义了符合 AgentExecutor 的 AutoGPT 智能体
